In [170]:
import pandas as pd
import math
import numpy as np

In [171]:
#Import set of generated answers for this set of users
df_data = pd.read_csv("SADD_Options_Selected.csv")
df_data.sort_values(by="Group")


,User_ID,Group,SADD1P1,SADD1P2,SADD1P3,SADD1P4,SADD1P5
10,5,0,3,1,1,1,NaN
11,2,0,3,1,2,1,NaN
12,3,0,3,1,2,2,NaN
13,4,0,3,1,2,2,NaN
14,1,0,3,2,3,3,4.0
5,9,1,3,1,1,1,NaN
6,10,1,3,1,2,1,NaN
7,7,1,3,1,2,1,NaN
8,8,1,3,2,3,2,1.0
9,6,1,3,2,3,2,2.0


In [172]:
def majority(x,df) : 
    return x > math.ceil(len(df)) / 2
def unanimity(x,df) :
    return x == len(df)

def are_all_options_selected(result_table) :
    for val in result_table.values() :
        if val == None :
            return False
    return True

def get_parameters_left(result_table) :
    parameters_left = []
    for key,val in result_table.items() :
        if val is None : 
            parameters_left.append(key)
    return parameters_left
  
def show_options_selected(result_table) :
    for key,val in result_table.items() :
        if (val is None) :
            print(f"Parameter {key} doesn't have a selected option yet")
        else :
            print(f"Parameter {key}, O{val} was selected")

def check_criteria(result_table,df_users,group_query,condition) :
    if not (are_all_options_selected(result_table)) :
        parameters_name = get_parameters_left(result_table)
        
        df_group = df_users.query(group_query)
        #For each parameter 
        for parameter_name in parameters_name :
                            
            #Get the values for this parameter
            parameter_values = df_group[parameter_name].value_counts()
            for index,param_value in parameter_values.items() :
                
                #Check for majority
                if condition == "majority" :
                    if majority(param_value,df_group):
                        result_table.update({parameter_name : index })
                    
                #Check for unanimity
                elif condition == "unanimity" :
                    if unanimity(param_value,df_group):
                        result_table.update({parameter_name : index })

def remove_linked_parameters(result_table,linked_parameters) :
    keys_to_remove = []
    for lnk_param in linked_parameters :
        for key,val in result_table.items() :
            if lnk_param[0] == key and lnk_param[1] != val :
                keys_to_remove.append(lnk_param[2])
    for key in keys_to_remove :
        print(f"Parameter {key} was removed as its linked parameter option was not selected.")
        result_table.pop(key)

result_table = dict.fromkeys(df_data.columns.copy()[2:])

linked_parameters = [["SADD1P3",3,"SADD1P5"]]

# First set of Resolution Strategies

# Unanimity for Expert and Advanced
check_criteria(result_table,df_data," Group == 1 | Group == 2","unanimity")

# If we still have options to resolve
if not (are_all_options_selected(result_table)) :
    # We apply the second strategy Unanimity for Expert
    check_criteria(result_table,df_data,"Group == 2","unanimity")

show_options_selected(result_table)


Parameter SADD1P1, O3 was selected
Parameter SADD1P2 doesn't have a selected option yet
Parameter SADD1P3 doesn't have a selected option yet
Parameter SADD1P4, O1 was selected
Parameter SADD1P5 doesn't have a selected option yet


In [173]:
def get_least_vote(threshold,df_data,val) :
    size = df_data[val].unique().size
    serie_opt = df_data[val].value_counts().reindex(range(1,size+1), fill_value=0)
    limit = serie_opt.sum()*threshold
    opt_to_remove = []
    for index, value in serie_opt.items() :
        if (value < limit) :
            opt_to_remove.append(index)
    return opt_to_remove

def remove_least_voted_options(df_data,result_table,threshold) :
    params = get_parameters_left(result_table)
    param_options_to_remove = {}
    for val in params :
        param_options_to_remove.update({val : get_least_vote(threshold,df_data,val)})
        
    return param_options_to_remove

# Reduction of parameters with least votes according to the criterias with a threshold of 15%
p_opt_to_rm = remove_least_voted_options(df_data,result_table,0.15)

print(p_opt_to_rm)


{'SADD1P2': [], 'SADD1P3': [1], 'SADD1P5': [3]}


In [174]:
# Loading the second questionnaire result
df_data_2 = pd.read_csv("SADD_Options_Selected_2.csv",sep=",")
df_data_2.sort_values(by="Group")

,User_ID,Group,SADD1P2,SADD1P3,SADD1P5
5,9,1,1,3,4.0
6,10,1,1,3,2.0
7,7,1,2,3,2.0
8,8,1,2,3,1.0
9,6,1,2,2,NaN
0,14,2,1,2,NaN
1,15,2,1,2,NaN
2,11,2,1,2,NaN
3,12,2,1,2,NaN
4,13,2,2,3,2.0


In [175]:
# Majority for Expert and Advanced
check_criteria(result_table,df_data_2," Group == 1 | Group == 2","majority")
print(result_table)
if not (are_all_options_selected(result_table)) :
    # Majority for Expert
    check_criteria(result_table,df_data,"Group == 2","majority")
remove_linked_parameters(result_table,linked_parameters)
show_options_selected(result_table)

{'SADD1P1': 3, 'SADD1P2': 1, 'SADD1P3': None, 'SADD1P4': 1, 'SADD1P5': None}
Parameter SADD1P5 was removed as its linked parameter option was not selected.
Parameter SADD1P1, O3 was selected
Parameter SADD1P2, O1 was selected
Parameter SADD1P3, O2 was selected
Parameter SADD1P4, O1 was selected
